In [1]:
import os
import numpy as np
import pandas as pd
import ast
import re
import csv

import nltk
#from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
ps = PorterStemmer()
lemma = WordNetLemmatizer()

from sklearn.feature_extraction.text import CountVectorizer


import gensim
from gensim import corpora, models
os.chdir('/home/ekhongl/Codes/DL - Topic Modelling')

In [68]:
dat = pd.read_csv('data/pef.csv', header = None,skip_blank_lines=False)
dat = list(dat[0])

str

In [85]:
vocab =  np.genfromtxt('data/dtm_2000_20news.csv', dtype=str, delimiter=',', max_rows = 1)[1:]
vocab

array(['edu', 'subject', 'com', ..., 'zero', 'aaron', 'philadelphia'], 
      dtype='<U13')

In [97]:
print('------- Data cleaning -------')                
stopwords_en = stopwords.words('english')
dat_clean = []
for i in range(len(dat)):
    
    if type(dat[i]) != float:
        ''' tokenization and punctuation removal '''
        # uses nltk tokenization - e.g. shouldn't = [should, n't] instead of [shouldn, 't]
        tmp_doc = nltk.tokenize.word_tokenize(dat[i].lower())

        # split words sperated by fullstops
        tmp_doc_split = [w.split('.') for w in tmp_doc if len(w.split('.')) > 1]
        # flatten list
        tmp_doc_split = [i_sublist for i_list in tmp_doc_split for i_sublist in i_list]
        # clean split words
        tmp_doc_split = [w for w in tmp_doc_split if re.search('^[a-z]+$',w)]

        # drop punctuations
        tmp_doc_clean = [w for w in tmp_doc if re.search('^[a-z]+$',w)]
        tmp_doc_clean.extend(tmp_doc_split)

        ''' stop word removal'''
        tmp_doc_clean_stop = [w for w in tmp_doc_clean if w not in stopwords_en]
        #retain only words with 2 characters or more
        tmp_doc_clean_stop = [w for w in  tmp_doc_clean_stop if len(w) >2]

        ''' stemming (using the Porter's algorithm)'''
        tmp_doc_clean_stop_stemmed = [ps.stem(w) for w in tmp_doc_clean_stop]
        dat_clean.append(tmp_doc_clean_stop_stemmed)
    else:
        dat_clean.append([])
    

------- Data cleaning -------


In [87]:
vec = CountVectorizer(input = 'content', lowercase = False, vocabulary = vocab)

In [91]:
np.array([] ).reshape(0,len(vocab))

array([], shape=(0, 2000), dtype=float64)

In [95]:
vec.fit_transform(dat_clean[i]).toarray().sum(axis=0, keepdims = True).shape

(1, 2000)

In [98]:
dtm = np.array([] ).reshape(0,len(vocab))
for i in range(len(dat_clean)):
    dtm = np.concatenate( (dtm, vec.fit_transform(dat_clean[i]).toarray().sum(axis=0, keepdims = True) ), axis = 0)

In [100]:
dtm.shape

(50, 2000)

In [120]:
from __future__ import print_function, division
import os
import sys
import timeit
from six.moves import cPickle as pickle

import numpy as np
import pandas as pd

import theano
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams

from lib.deeplearning import deepbeliefnet

os.chdir('/home/ekhongl/Codes/DL - Topic Modelling')

In [123]:
model = deepbeliefnet( architecture = [2000, 500, 500, 128], n_outs=20)

... building the model
Building layer: 0
   Input units: 2000
  Output units: 500
Building layer: 1
   Input units: 500
  Output units: 500
Building layer: 2
   Input units: 500
  Output units: 128


In [ ]:
'params_2000/dbn_params_dropout'

In [116]:
test_input = theano.shared(dtm.astype('float32'))

In [117]:
output = model.score(input = test_input)

In [118]:
output

array([[  2.54660677e-02,   8.91028285e-01,   7.98969090e-01, ...,
          5.59706299e-04,   1.12502035e-02,   8.59574199e-01],
       [  1.83742642e-02,   8.55456412e-01,   8.70182037e-01, ...,
          6.76897645e-04,   8.63240100e-03,   8.46488357e-01],
       [  1.21897878e-02,   8.65011692e-01,   9.73743439e-01, ...,
          1.10489945e-03,   6.92097377e-03,   9.22724724e-01],
       ..., 
       [  1.15953190e-02,   8.70056152e-01,   9.69405472e-01, ...,
          1.02550129e-03,   6.73701521e-03,   9.29384947e-01],
       [  1.92849617e-02,   8.88312459e-01,   8.48161757e-01, ...,
          5.93443285e-04,   1.23798046e-02,   8.25212061e-01],
       [  2.91127190e-02,   9.16935563e-01,   7.41483986e-01, ...,
          6.81867590e-04,   1.28407655e-02,   7.49706447e-01]], dtype=float32)